In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load Kaggle dataset
kaggle_df = pd.read_csv(r"C:\Users\ASUS\Downloads\KaggleV2-May-2016.csv", parse_dates=['ScheduledDay', 'AppointmentDay'])

# Load Clinic dataset (assuming you've extracted from PDF to CSV)
clinic_df = pd..read_excel(r"C:\Users\ASUS\Downloads\Clinic Data 2022-2024.xlsx")

In [23]:
import pandas as pd

def clean_kaggle_data(df):
    """
    Clean and preprocess the Kaggle medical appointments dataset
    Returns a cleaned copy of the DataFrame
    """
    # Create a working copy
    df_clean = df.copy()
    
    # Debug: Check column types before conversion
    print("Before conversion:")
    print(f"ScheduledDay type: {type(df_clean['ScheduledDay'].iloc[0]) if 'ScheduledDay' in df_clean.columns else 'Column not found'}")
    print(f"AppointmentDay type: {type(df_clean['AppointmentDay'].iloc[0]) if 'AppointmentDay' in df_clean.columns else 'Column not found'}")
    
    # Robust datetime conversion with error handling
    def convert_to_datetime(series):
        try:
            # First try pandas to_datetime
            result = pd.to_datetime(series, errors='coerce')
            if result.isna().any():
                # If conversion fails for some values, try parsing specific formats
                for fmt in ['%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M', '%Y-%m-%d', '%m/%d/%Y']:
                    try:
                        result = pd.to_datetime(series, format=fmt, errors='coerce')
                        if not result.isna().all():
                            break
                    except:
                        continue
            return result
        except Exception as e:
            print(f"Error converting datetime: {e}")
            return series
    
    # Apply datetime conversion only if columns exist
    if 'ScheduledDay' in df_clean.columns:
        df_clean['ScheduledDay'] = convert_to_datetime(df_clean['ScheduledDay'])
    if 'AppointmentDay' in df_clean.columns:
        df_clean['AppointmentDay'] = convert_to_datetime(df_clean['AppointmentDay'])
    
    # Verify conversion worked for existing columns
    if 'ScheduledDay' in df_clean.columns and not pd.api.types.is_datetime64_any_dtype(df_clean['ScheduledDay']):
        raise ValueError("Failed to convert ScheduledDay to datetime")
    if 'AppointmentDay' in df_clean.columns and not pd.api.types.is_datetime64_any_dtype(df_clean['AppointmentDay']):
        raise ValueError("Failed to convert AppointmentDay to datetime")
    
    # Handle Age - remove negative values and impute median
    if 'Age' in df_clean.columns:
        df_clean = df_clean[df_clean['Age'] >= 0].copy()
        median_age = df_clean['Age'].median()
        df_clean['Age'] = df_clean['Age'].fillna(median_age)
    
    # Standardize No-show column if exists
    if 'No-show' in df_clean.columns:
        df_clean['No-show'] = df_clean['No-show'].str.strip().str.title()
    
    # Convert binary columns to 0/1
    binary_cols = ['Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'SMS_received']
    for col in binary_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(int)
    
    # Standardize gender if exists
    if 'Gender' in df_clean.columns:
        df_clean['Gender'] = df_clean['Gender'].str.strip().str.upper()
    
    # Calculate days between scheduling and appointment (FIXED VERSION)
    if 'ScheduledDay' in df_clean.columns and 'AppointmentDay' in df_clean.columns:
        # Normalize both dates to remove time components
        scheduled_date = df_clean['ScheduledDay'].dt.normalize()
        appointment_date = df_clean['AppointmentDay'].dt.normalize()
        
        # Calculate difference in calendar days
        df_clean['DaysBetween'] = (appointment_date - scheduled_date).dt.days
        
        # Remove records with impossible negative days (data errors)
        df_clean = df_clean[df_clean['DaysBetween'] >= 0]
    
    # Extract datetime features if columns exist
    if 'AppointmentDay' in df_clean.columns:
        df_clean['AppointmentDOW'] = df_clean['AppointmentDay'].dt.day_name()
        df_clean['AppointmentMonth'] = df_clean['AppointmentDay'].dt.month_name()
    if 'ScheduledDay' in df_clean.columns:
        df_clean['ScheduledHour'] = df_clean['ScheduledDay'].dt.hour
    
    # Create age groups if Age exists
    if 'Age' in df_clean.columns:
        bins = [0, 18, 35, 55, 75, 120]
        labels = ['0-18', '19-35', '36-55', '56-75', '75+']
        df_clean['AgeGroup'] = pd.cut(df_clean['Age'], bins=bins, labels=labels, right=False)
    
    # Drop irrelevant columns if they exist
    cols_to_drop = ['Handcap', 'Neighbourhood']
    df_clean = df_clean.drop(columns=[col for col in cols_to_drop if col in df_clean.columns])
    
    return df_clean

# First load your data with proper datetime parsing
try:
    kaggle_df = pd.read_csv(
        r"C:\Users\ASUS\Downloads\KaggleV2-May-2016.csv",
        parse_dates=['ScheduledDay', 'AppointmentDay'],
        encoding='latin1'
    )
except Exception as e:
    print(f"Error loading CSV: {e}")
    # Try without parse_dates if that fails
    kaggle_df = pd.read_csv(
        r"C:\Users\ASUS\Downloads\KaggleV2-May-2016.csv",
        encoding='latin1'
    )

# Now clean the data
kaggle_df_clean = clean_kaggle_data(kaggle_df)

# Verify results
print("\nCleaning completed successfully!")
print(kaggle_df_clean.info())
print("\nDaysBetween distribution:")
print(kaggle_df_clean['DaysBetween'].describe())
print("\nFirst 5 records:")
print(kaggle_df_clean[['ScheduledDay', 'AppointmentDay', 'DaysBetween']].head())

Before conversion:
ScheduledDay type: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
AppointmentDay type: <class 'pandas._libs.tslibs.timestamps.Timestamp'>

Cleaning completed successfully!
<class 'pandas.core.frame.DataFrame'>
Index: 110521 entries, 0 to 110526
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   PatientId         110521 non-null  float64            
 1   AppointmentID     110521 non-null  int64              
 2   Gender            110521 non-null  object             
 3   ScheduledDay      110521 non-null  datetime64[ns, UTC]
 4   AppointmentDay    110521 non-null  datetime64[ns, UTC]
 5   Age               110521 non-null  int64              
 6   Scholarship       110521 non-null  int32              
 7   Hipertension      110521 non-null  int32              
 8   Diabetes          110521 non-null  int32              
 9   Alcoholism        110521 non-null 

In [41]:
import pandas as pd

def clean_clinic_data(df):
    """
    Clean and preprocess the clinic appointments dataset
    Returns a cleaned DataFrame with proper types and calculations
    """
    # Create a working copy
    df_clean = df.copy()
    
    # 1. Find where the actual data starts (first row with numeric Account)
    data_start = None
    for i, row in df_clean.iterrows():
        if str(row.iloc[0]).strip().replace('.','').isdigit():  # More flexible numeric check
            data_start = i
            break
    
    if data_start is None:
        raise ValueError("Could not find start of data in the file")
    
    # 2. Re-read with proper headers (skip the metadata rows)
    df_clean = pd.read_excel(
        r"C:\Users\ASUS\Downloads\Clinic Data 2022-2024.xlsx",
        header=None,
        skiprows=data_start
    )
    
    # 3. Set proper column names
    column_names = [
        'Account',
        'AppointmentDate',
        'Status',
        'Code',
        'Provider',
        'Message',
        'ContactDate'
    ]
    df_clean.columns = column_names
    
    # 4. Remove any remaining non-data rows (where Account isn't numeric)
    df_clean = df_clean[df_clean['Account'].astype(str).str.strip().str.replace('.','').str.isdigit()]
    
    # 5. Convert Account to integer safely
    df_clean['Account'] = pd.to_numeric(df_clean['Account'], errors='coerce').astype('Int64')
    
    # 6. Convert dates with explicit formats
    for col in ['AppointmentDate', 'ContactDate']:
        df_clean[col] = pd.to_datetime(
            df_clean[col],
            format='mixed',
            dayfirst=False,
            yearfirst=True,
            errors='coerce'
        )
    
    # 7. Clean and standardize data
    severity_map = {
        '6MR': 'Mild', '12MR': 'Mild', 'SCAL': 'Mild', 'NPC': 'Mild', 'NP': 'Mild',
        'COMP': 'Moderate', 'RCT': 'Moderate', 'EXO': 'Moderate', 'CRN': 'Moderate', 
        'DENTR': 'Moderate', 'PAIN': 'Severe', 'EMER': 'Severe', 'EXTR': 'Severe', 
        'RAD': 'Severe', 'NGI': 'Severe', 'FSCAL': 'Mild', 'CF': 'Moderate'
    }
    
    # Standardize status
    df_clean['CancellationType'] = (
        df_clean['Status']
        .astype(str)
        .str.strip()
        .str.lower()
        .apply(lambda x: 
            'No-Show' if 'no-show' in x else
            'ShortNotice' if 'short notice cancel' in x else
            'Other'
        )
    )
    
    # Extract LVM flag
    df_clean['LVM_Flag'] = (
        df_clean['Message']
        .astype(str)
        .str.contains(r'LVM TO BK APPT', case=False, na=False)
        .astype(int)
    )
    
    # Map severity
    df_clean['Code'] = df_clean['Code'].astype(str).str.strip().str.upper()
    df_clean['Severity'] = df_clean['Code'].map(severity_map).fillna('Unknown')
    
    # Calculate days warning (absolute value)
    df_clean['DaysWarning'] = (
        (df_clean['AppointmentDate'] - df_clean['ContactDate'])
        .dt.days
        .abs()
        .fillna(0)
        .astype(int)
    )
    
    # 8. Final cleaning
    df_clean = df_clean.dropna(subset=['Code', 'Status'])
    df_clean = df_clean.reset_index(drop=True)
    
    return df_clean

# Load and process data
try:
    print("Loading data...")
    clinic_df = pd.read_excel(r"C:\Users\ASUS\Downloads\Clinic Data 2022-2024.xlsx")
    
    print("Cleaning data...")
    clinic_df_clean = clean_clinic_data(clinic_df)
    
    print("\n=== FINAL CLEANED DATA ===")
    print(f"Rows: {len(clinic_df_clean)}, Columns: {len(clinic_df_clean.columns)}")
    
    print("\nDATA TYPES:")
    print(clinic_df_clean.dtypes)
    
    print("\nSAMPLE DATA:")
    print(clinic_df_clean.head(3).to_dict('records'))  # Show as dict for better readability
    
    print("\nSEVERITY DISTRIBUTION:")
    print(clinic_df_clean['Severity'].value_counts())
    
    print("\nCANCELLATION TYPE DISTRIBUTION:")
    print(clinic_df_clean['CancellationType'].value_counts())
    
    print("\nDAYS WARNING STATISTICS:")
    print(clinic_df_clean['DaysWarning'].describe())
    
except Exception as e:
    print(f"\nERROR: {str(e)}")
    print("\nDebug Info:")
    if 'clinic_df' in locals():
        print("Raw data columns:", clinic_df.columns.tolist())
        print("First 5 rows:", clinic_df.head().to_dict('records'))

Loading data...
Cleaning data...

=== FINAL CLEANED DATA ===
Rows: 1137, Columns: 11

DATA TYPES:
Account                      Int64
AppointmentDate     datetime64[ns]
Status                      object
Code                        object
Provider                    object
Message                     object
ContactDate         datetime64[ns]
CancellationType            object
LVM_Flag                     int32
Severity                    object
DaysWarning                  int32
dtype: object

SAMPLE DATA:
[{'Account': 332, 'AppointmentDate': Timestamp('2015-03-19 00:00:00'), 'Status': 'No-Show', 'Code': '12MR', 'Provider': '2CM', 'Message': 'APR 3, 2019 LVM TO BK APPT-DK MAR 8, 201', 'ContactDate': Timestamp('2019-04-03 00:00:00'), 'CancellationType': 'No-Show', 'LVM_Flag': 1, 'Severity': 'Mild', 'DaysWarning': 1476}, {'Account': 341, 'AppointmentDate': Timestamp('2021-11-01 00:00:00'), 'Status': 'Short Notice Cancel.', 'Code': 'FSCAL', 'Provider': '2MP', 'Message': 'OCT 18, 22 LVM TO 

In [43]:
print(clinic_df_clean.nlargest(5, 'DaysWarning')[['AppointmentDate','ContactDate','DaysWarning']])

    AppointmentDate ContactDate  DaysWarning
226      2004-04-29  2017-10-03         4905
71       2009-06-09  2018-12-17         3478
192      2009-11-16  2019-02-27         3390
316      2010-06-07  2002-01-03         3077
261      2011-12-29  2020-01-16         2940


In [45]:

# Remove records with DaysWarning > 3 years (1095 days)
clinic_df_clean = clinic_df_clean[clinic_df_clean['DaysWarning'] <= 1095]


In [47]:
print(clinic_df_clean.head())

   Account AppointmentDate                Status   Code Provider  \
1      341      2021-11-01  Short Notice Cancel.  FSCAL      2MP   
3      357      2018-06-12  Short Notice Cancel.    ADJ        1   
5      357      2017-05-17  Short Notice Cancel.    RCT        1   
7      433      2025-01-23  Short Notice Cancel.    4MR      2SP   
8      464      2025-01-02  Short Notice Cancel.   SCAL      2MP   

                                     Message ContactDate CancellationType  \
1  OCT 18, 22 LVM TO BK APPT, NANCY\nJune 25  2022-10-18      ShortNotice   
3    Short Notice Cancellation-[11 Jun,18 12  2018-11-27      ShortNotice   
5   APRIL 30 2018 CALLED NO ANSWER/FR    aug  2018-04-30      ShortNotice   
7  APR 24, 2025 PT IS SICK, WCB TO BK LATER.  2025-04-24      ShortNotice   
8    APR 24, 2025 LVM TO BK APPT. AISHA (JAN  2025-04-24      ShortNotice   

   LVM_Flag  Severity  DaysWarning  
1         1      Mild          351  
3         0   Unknown          168  
5         0  Mode

In [49]:
print(kaggle_df_clean.head())

      PatientId  AppointmentID Gender              ScheduledDay  \
0  2.987250e+13        5642903      F 2016-04-29 18:38:08+00:00   
1  5.589978e+14        5642503      M 2016-04-29 16:08:27+00:00   
2  4.262962e+12        5642549      F 2016-04-29 16:19:04+00:00   
3  8.679512e+11        5642828      F 2016-04-29 17:29:31+00:00   
4  8.841186e+12        5642494      F 2016-04-29 16:07:23+00:00   

             AppointmentDay  Age  Scholarship  Hipertension  Diabetes  \
0 2016-04-29 00:00:00+00:00   62            0             1         0   
1 2016-04-29 00:00:00+00:00   56            0             0         0   
2 2016-04-29 00:00:00+00:00   62            0             0         0   
3 2016-04-29 00:00:00+00:00    8            0             0         0   
4 2016-04-29 00:00:00+00:00   56            0             1         1   

   Alcoholism  SMS_received No-show  DaysBetween AppointmentDOW  \
0           0             0      No            0         Friday   
1           0           

In [51]:
# Save cleaned datasets
kaggle_df_clean.to_csv('cleaned_kaggle_data.csv', index=False)
clinic_df_clean.to_csv('cleaned_clinic_data.csv', index=False)